<a href="https://colab.research.google.com/github/woodword-0/ML-Projects/blob/main/DeepDream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install image
!pip install pillow

In [ ]:
import tensorflow as tf
import numpy as np
import IPython.display as display
# import PIL.image
from tensorflow.keras.preprocessing import image

In [ ]:
import PIL
from PIL import Image
print('Pillow Version:', PIL.__version__)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
im = Image.open('/Ellie.png')
rgb_im = im.convert('RGB')
rgb_im.save('Ellie.jpg')

In [ ]:
url = '/Ellie.png'


In [ ]:
PIL.Image.open(url)

In [ ]:
#Downloading an image and read it into a numpy array
def download(url,max_dim = None):
  name =  url.split('/')[-1]
  image_path = '/content/Ellie.jpg'
  img = PIL.Image.open(image_path)
  if max_dim:
    img.thumbnail((max_dim, max_dim))
  return np.array(img)
#Normalize an image
def deprocess(img):
  img = 255*(img + 1.0)/2.0
  return tf.cast(img, tf.uint8)
#Display an image
def show(img):
  display.display(PIL.Image.fromarray(np.array(img)))
#Downsizing the image making it easier to work with
original_img = download(url, max_dim = 500)
show(original_img)
display.display(url)

Prepare the feature extraction model
We will download and prepare a pretrained classification model InceptionV3

In [ ]:
base_model  = tf.keras.applications.InceptionV3(include_top= False, weights = 'imagenet')

In [ ]:
#Maximize the activation of these layers
names = ['mixed4', 'mixed6'] #mixed3 mixed 5
layers = [base_model.get_layer(name).output for name in names]
#Create the feature extraction model
dream_model = tf.keras.Model(inputs = base_model.input, outputs = layers)

Calculate Loss: Loss is the sum of the activations in the chosen layers. We want to maximize loss here

In [ ]:
def calc_loss(img, model):
  #Pass forward the image through the model to retrieve the activations
  #Converts the image into a batch of size 1
  img_batch = tf.expand_dims(img, axis = 0)
  layer_activations = model(img_batch)
  if len(layer_activations) == 1:
    layer_activations = [layer_activations]
  losses = []
  for act in layer_activations:
    loss = tf.math.reduce_mean(act)
    losses.append(loss)
  return tf.reduce_sum(losses)

Gradient Ascent: After calculating loss, we now just need to calculate gradients with respect to the image and add to the original image. Adding gradients enhances the patterns seen by the network 

In [ ]:
class DeepDream(tf.Module):
  def __init__(self, model):
    self.model = model
  @tf.function(
      input_signature = (
          tf.TensorSpec(shape = [None, None, 3], dtype = tf.float32), 
          tf.TensorSpec(shape = [], dtype = tf.int32), 
          tf.TensorSpec(shape = [], dtype = tf.float32))
  )
  def __call__(self, img, steps, step_size):
    print('Tracing')
    loss = tf.constant(0.0)
    for n in tf.range(steps):
      with tf.GradientTape() as tape:
        #This needs gradients relative to the image
        #GradientTape only watches tf.Variables 
        tape.watch(img)
        loss = calc_loss(img, self.model)
      #Calculate the gradient of the loss with respect to the pixels of the input image
      gradients = tape.gradient(loss, img)
      #Normalize the gradients
      gradients /= tf.math.reduce_std(gradients) + 1e-8 #computes std of tensor elements
      #In gradient ascent, the loss is maximized so that the input image
      #increasingly excites the layers
      #We can update the image by directly adding the gradients
      #(since they are the same shape)
      img = img + gradients*step_size
      img = tf.clip_by_value(img, -1, 1) #Clips tensor values to a specified min and max.
    return loss, img

  

In [ ]:
deepdream = DeepDream(dream_model)

Main Loop

In [ ]:
def run_deep_dream_simple(img, steps = 100, step_size = 0.01):
  #convert from uint8 to the range expected by the model.
  img = tf.keras.applications.inception_v3.preprocess_input(img)
  img = tf.convert_to_tensor(img)
  step_size = tf.convert_to_tensor(step_size)
  steps_remaining  = steps
  step = 0
  while steps_remaining:
    if steps_remaining > 100:
      run_steps = tf.constant(100)
    else:
      run_steps = tf.constant(steps_remaining)
    steps_remaining -= run_steps
    step += run_steps

    loss, img = deepdream(img, run_steps, tf.constant(step_size))
    display.clear_output(wait = True)
    show(deprocess(img))
    print("Step {}, loss {}".format(step, loss))
  result = deprocess(img)
  display.clear_output(wait = True)
  show(result)

  return result

In [ ]:
dream_img = run_deep_dream_simple(img = original_img, steps = 100, step_size = 0.01)

Output is noisy
Image is Low resolution
Patterns appear all a the same grainularity
To solve these problems, apply gradient ascent at different scales

In [ ]:
import time
start = time.time()
OCTAVE_SCALE = 1.30
img = tf.constant(np.array(original_img))
base_shape = tf.shape(img)[:-1]
float_base_shape = tf.cast(base_shape, tf.float32)
for n in range(-2,3):
  new_shape = tf.cast(float_base_shape*(OCTAVE_SCALE**n), tf.int32)
  img = tf.image.resize(img, new_shape).numpy()
  img = run_deep_dream_simple(img = img, steps = 50, step_size=0.01)

display.clear_output(wait = True)
img = tf.image.resize(img, base_shape)
img = tf.image.convert_image_dtype(img/255.0, dtype=tf.uint8)
show(img)

end = time.time()
end-start

To deal with large images, we split the image into windows and perform a gradient on each tile

In [ ]:
def random_roll(img, maxroll):
  #randomly shift the image to avoid tiled boundaries
  shift = tf.random.uniform(shape = [2], minval =- maxroll, maxval = maxroll, dtype = tf.int32)
  img_rolled = tf.roll(img, shift = shift, axis = [0,1])
  return shift, img_rolled

In [ ]:
shift, img_rolled = random_roll(np.array(original_img), 512)
show(img_rolled)

A tiled equivalent of the deepdream function

In [ ]:
class TiledGradients(tf.Module):
  def __init__(self, model):
    self.model = model

  @tf.function(
      input_signature=(
        tf.TensorSpec(shape=[None,None,3], dtype=tf.float32),
        tf.TensorSpec(shape=[], dtype=tf.int32),)
  )
  def __call__(self, img, tile_size=512):
    shift, img_rolled = random_roll(img, tile_size)

    # Initialize the image gradients to zero.
    gradients = tf.zeros_like(img_rolled)

    # Skip the last tile, unless there's only one tile.
    xs = tf.range(0, img_rolled.shape[0], tile_size)[:-1]
    if not tf.cast(len(xs), bool):
      xs = tf.constant([0])
    ys = tf.range(0, img_rolled.shape[1], tile_size)[:-1]
    if not tf.cast(len(ys), bool):
      ys = tf.constant([0])

    for x in xs:
      for y in ys:
        # Calculate the gradients for this tile.
        with tf.GradientTape() as tape:
          # This needs gradients relative to `img_rolled`.
          # `GradientTape` only watches `tf.Variable`s by default.
          tape.watch(img_rolled)

          # Extract a tile out of the image.
          img_tile = img_rolled[x:x+tile_size, y:y+tile_size]
          loss = calc_loss(img_tile, self.model)

        # Update the image gradients for this tile.
        gradients = gradients + tape.gradient(loss, img_rolled)

    # Undo the random shift applied to the image and its gradients.
    gradients = tf.roll(gradients, shift=-shift, axis=[0,1])

    # Normalize the gradients.
    gradients /= tf.math.reduce_std(gradients) + 1e-8 

    return gradients

In [ ]:
get_tiled_gradients = TiledGradients(dream_model)

Putting this all together gives a scalable, octave-aware deepdream implementation

In [ ]:
def run_deep_dream_with_octaves(img, steps_per_octave=100, step_size=0.01, 
                                octaves=range(-2,3), octave_scale=1.3):
  base_shape = tf.shape(img)
  img = tf.keras.preprocessing.image.img_to_array(img)
  img = tf.keras.applications.inception_v3.preprocess_input(img)

  initial_shape = img.shape[:-1]
  img = tf.image.resize(img, initial_shape)
  for octave in octaves:
    # Scale the image based on the octave
    new_size = tf.cast(tf.convert_to_tensor(base_shape[:-1]), tf.float32)*(octave_scale**octave)
    img = tf.image.resize(img, tf.cast(new_size, tf.int32))

    for step in range(steps_per_octave):
      gradients = get_tiled_gradients(img)
      img = img + gradients*step_size
      img = tf.clip_by_value(img, -1, 1)

      if step % 10 == 0:
        display.clear_output(wait=True)
        show(deprocess(img))
        print ("Octave {}, Step {}".format(octave, step))

  result = deprocess(img)
  return result

In [ ]:
img = run_deep_dream_with_octaves(img=original_img, step_size=0.01)

display.clear_output(wait=True)
img = tf.image.resize(img, base_shape)
img = tf.image.convert_image_dtype(img/255.0, dtype=tf.uint8)
show(img)